For this project you'll dig into a large amount of text and apply most of what you've covered in this unit and in the course so far.

First, pick a set of texts. This can be either a series of novels, chapters, or articles. Anything you'd like. It just has to have multiple entries of varying characteristics. At least 100 should be good. There should also be at least 10 different authors, but try to keep the texts related (either all on the same topic of from the same branch of literature - something to make classification a bit more difficult than obviously different subjects).

This capstone can be an extension of your NLP challenge if you wish to use the same corpus. If you found problems with that data set that limited your analysis, however, it may be worth using what you learned to choose a new corpus. Reserve 25% of your corpus as a test set.

The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

Lastly return to your holdout group. Does your clustering on those members perform as you'd expect? Have your clusters remained stable or changed dramatically? What about your model? Is it's performance consistent?

If there is a divergence in the relative stability of your model and your clusters, delve into why.

Your end result should be a write up of how clustering and modeling compare for classifying your texts. What are the advantages of each? Why would you want to use one over the other? Approximately 3-5 pages is a good length for your write up, and remember to include visuals to help tell your story!

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

from collections import Counter

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import spacy
nlp = spacy.load('en')

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

## Load files

In [5]:
filename=['brady.txt','butler.txt','cutting.txt','dawson.txt','dyke.txt',
          'hughes.txt','mitchell.txt','richmond.txt','rinehart.txt','stretton.txt']


In [6]:
def load_text(filename):
    file=open(filename,'r')
    text=file.read()
    file.close
    text=nlp(text)
    return text

In [7]:
sentences=pd.DataFrame()
for i in range(0,len(filename)):
    text=load_text(filename[i])
    author=filename[i].replace('.txt','')
    text_sents=[[sent,author] for sent in text.sents]
    text_df=pd.DataFrame(text_sents)
    sentences=sentences.append(text_df)

In [8]:
sentences

,0,1
0,"(ï»¿The, Project, Gutenberg, eBook, ,, And, Th...",brady
1,"(Illustrated, by, Walter, B., Everett, \n\n\n)",brady
2,"(This, eBook, is, for, the, use, of, anyone, a...",brady
3,"(You, may, copy, it, ,, give, it, away, or, \n...",brady
4,"(Title, :)",brady
5,"(And, Thus, He, Came, \n\n)",brady
6,"(Author, :, Cyrus, Townsend, Brady, \n\n, Rele...",brady
7,"(:, January, 5, ,, 2005, )",brady
8,"([, eBook, #, 14606, ], \n\n)",brady
9,"(Language, :)",brady


In [9]:
# Utility function to create a list of the 10 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(10)]

In [10]:
common_words=set()
for i in range(0,len(filename)):
    text=load_text(filename[i])
    words=bag_of_words(text)
    common_words.update(words)

In [11]:
# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(index=range(len(sentences)),columns=common_words)
    print(df)
    df['text_sentence'] = sentences[0].values
    print(df)
    df['text_source'] = sentences[1].values
    print(df)
    df.loc[:,common_words] = 0
    
    print()
    print(df)
    print(df.shape)
    
    print(df['text_source'])
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
       
    return df

In [12]:
type(common_words)

set

In [13]:
len(common_words)

22

In [14]:
type(sentences)

pandas.core.frame.DataFrame

In [15]:
len(sentences)

7224

In [16]:
word_counts = bow_features(sentences, common_words)

     mrs.  say  god gratz  man \n      rhoda work joan chicken    ...      be  \
0     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN    ...     NaN   
1     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN    ...     NaN   
2     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN    ...     NaN   
3     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN    ...     NaN   
4     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN    ...     NaN   
5     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN    ...     NaN   
6     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN    ...     NaN   
7     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN    ...     NaN   
8     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN    ...     NaN   
9     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN    ...     NaN   
10    NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN    ...     NaN   
11    NaN  NaN  NaN   NaN  N

[7224 rows x 22 columns]
     mrs.  say  god gratz  man \n      rhoda work joan chicken  \
0     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN   
1     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN   
2     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN   
3     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN   
4     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN   
5     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN   
6     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN   
7     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN   
8     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN   
9     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN   
10    NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN   
11    NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN   
12    NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN   
13    NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  

[7224 rows x 23 columns]
     mrs.  say  god gratz  man \n      rhoda work joan chicken     ...      \
0     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN     ...       
1     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN     ...       
2     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN     ...       
3     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN     ...       
4     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN     ...       
5     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN     ...       
6     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN     ...       
7     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN     ...       
8     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN     ...       
9     NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN     ...       
10    NaN  NaN  NaN   NaN  NaN     NaN   NaN  NaN  NaN     NaN     ...       
11    NaN  NaN  NaN   NaN  NaN     NaN 

[7224 rows x 24 columns]

     mrs. say god gratz man \n      rhoda work joan chicken     ...     the  \
0       0   0   0     0   0       0     0    0    0       0     ...       0   
1       0   0   0     0   0       0     0    0    0       0     ...       0   
2       0   0   0     0   0       0     0    0    0       0     ...       0   
3       0   0   0     0   0       0     0    0    0       0     ...       0   
4       0   0   0     0   0       0     0    0    0       0     ...       0   
5       0   0   0     0   0       0     0    0    0       0     ...       0   
6       0   0   0     0   0       0     0    0    0       0     ...       0   
7       0   0   0     0   0       0     0    0    0       0     ...       0   
8       0   0   0     0   0       0     0    0    0       0     ...       0   
9       0   0   0     0   0       0     0    0    0       0     ...       0   
10      0   0   0     0   0       0     0    0    0       0     ...       0   
11      0   0   0     0   

[7224 rows x 24 columns]
(7224, 24)
0          brady
1          brady
2          brady
3          brady
4          brady
5          brady
6          brady
7          brady
8          brady
9          brady
10         brady
11         brady
12         brady
13         brady
14         brady
15         brady
16         brady
17         brady
18         brady
19         brady
20         brady
21         brady
22         brady
23         brady
24         brady
25         brady
26         brady
27         brady
28         brady
29         brady
          ...   
7194    stretton
7195    stretton
7196    stretton
7197    stretton
7198    stretton
7199    stretton
7200    stretton
7201    stretton
7202    stretton
7203    stretton
7204    stretton
7205    stretton
7206    stretton
7207    stretton
7208    stretton
7209    stretton
7210    stretton
7211    stretton
7212    stretton
7213    stretton
7214    stretton
7215    stretton
7216    stretton
7217    stretton
7218    stretton
7219    stre

In [17]:
x

NameError: name 'x' is not defined

In [ ]:
df.shape

In [ ]:
'[1075]' in common_words

In [ ]:
'[1075]' in sentences

In [ ]:
len

In [ ]:
def text_cleaner(text):
    text=text.replace('gutenberg','')
    text=re.sub(r'ï»¿','',text)
    text=re.sub(r'\n','',text)
    text=re.sub('\[[^()]*\]','',text)
    text=re.sub('\([^()]*\)','',text)
    text=re.sub('[^\w\.\,]',' ',text)
    
    text=' '.join(text.split())
    text=text.split()
    
    return text

In [ ]:
df = pd.DataFrame()

for i in range(len(filename)):
    text = text_creater(filename[i])
    text_clean = text_cleaner(text)
    
    text_doc=nlp(text_clean)
    text_words=bag_of_words(text_doc)
    
    

In [ ]:
df.shape

In [ ]:
from sklearn.preprocessing import normalize
x=df.drop(['author'],axis=1)
x=normalize(x)
y=df['author']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y,
                                                    test_size=0.25,
                                                    random_state=0)

In [ ]:
k=10
model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
model.fit(x_train)

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
from sklearn import metrics
    
metrics.adjusted_rand_score(y_test, y_pred)

In [ ]:
y_pred=model.predict(x_train)
metrics.adjusted_rand_score(y_train, y_pred)

In [ ]:
from sklearn.metrics import pairwise_distances

model.fit(x)
labels=model.labels_

score=metrics.silhouette_score(x, labels, metric='euclidean')
print(score)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

lr = LogisticRegressionCV(cv=4)
lr.fit(x,y)
lr.score(x,y)

In [ ]:
from sklearn import ensemble
rfc = ensemble.RandomForestClassifier()
train = rfc.fit(x_train, y_train)

print('Training set score:', rfc.score(x_train, y_train))
print('\nTest set score:', rfc.score(x_test, y_test))